# Big Data and Data Analytics for Managers 

## Merlin John - 015027

Given the personal details of a patient, predict medical cost

Columns
- age: age of primary beneficiary
- sex: insurance contractor gender, female, male
- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
- children: Number of children covered by health insurance / Number of dependents
- smoker: Smoking
- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
- harges: Individual medical costs billed by health insurance

1. Start Hadoop allstart.sh
2. start spark--> pysparknb
3. hadoop fs -put /home/ashok/Downloads/insurance.csv

In [1]:
import numpy as np

from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml import Pipeline

#ML Logisitic Regression
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import time


In [2]:
df=spark.read.csv(path="insurance.csv", header=True, inferSchema=True)

In [3]:
df.show()

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|
| 23|  male|  34.4|       0|    no|southwest|   1826.843|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|
| 27|  male| 4

## Data Exploration

In [4]:
df.columns

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']

In [5]:
len(df.columns)

7

In [8]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [9]:
df.head(2)

[Row(age=19, sex='female', bmi=27.9, children=0, smoker='yes', region='southwest', charges=16884.924),
 Row(age=18, sex='male', bmi=33.77, children=1, smoker='no', region='southeast', charges=1725.5523)]

In [10]:
df.select("charges").show(10)

+-----------+
|    charges|
+-----------+
|  16884.924|
|  1725.5523|
|   4449.462|
|21984.47061|
|  3866.8552|
|  3756.6216|
|  8240.5896|
|  7281.5056|
|  6406.4107|
|28923.13692|
+-----------+
only showing top 10 rows



In [11]:
df.count()

1338

In [12]:
df.groupby('region').count().show()

+---------+-----+
|   region|count|
+---------+-----+
|northwest|  325|
|southeast|  364|
|northeast|  324|
|southwest|  325|
+---------+-----+



In [13]:
df.groupby('age').count().show()

+---+-----+
|age|count|
+---+-----+
| 31|   27|
| 53|   28|
| 34|   26|
| 28|   28|
| 27|   28|
| 26|   28|
| 44|   27|
| 22|   28|
| 47|   29|
| 52|   29|
| 40|   27|
| 20|   29|
| 57|   26|
| 54|   28|
| 48|   29|
| 19|   68|
| 64|   22|
| 41|   27|
| 43|   27|
| 37|   25|
+---+-----+
only showing top 20 rows



In [15]:
df.select(["age","charges"]).show()

+---+-----------+
|age|    charges|
+---+-----------+
| 19|  16884.924|
| 18|  1725.5523|
| 28|   4449.462|
| 33|21984.47061|
| 32|  3866.8552|
| 31|  3756.6216|
| 46|  8240.5896|
| 37|  7281.5056|
| 37|  6406.4107|
| 60|28923.13692|
| 25|  2721.3208|
| 62| 27808.7251|
| 23|   1826.843|
| 56| 11090.7178|
| 27| 39611.7577|
| 19|   1837.237|
| 52| 10797.3362|
| 23| 2395.17155|
| 56|  10602.385|
| 30|  36837.467|
+---+-----------+
only showing top 20 rows



In [14]:
df.describe().show()

+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|summary|               age|   sex|               bmi|         children|smoker|   region|           charges|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|  count|              1338|  1338|              1338|             1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  null|30.663396860986538|  1.0949177877429|  null|     null|13270.422265141257|
| stddev|14.049960379216147|  null| 6.098186911679012|1.205492739781914|  null|     null|12110.011236693992|
|    min|                18|female|             15.96|                0|    no|northeast|         1121.8739|
|    max|                64|  male|             53.13|                5|   yes|southwest|       63770.42801|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+



In [20]:
df.corr('age','charges')

0.299008193330648

In [21]:

vc=VectorAssembler(
    inputCols=['age',"bmi",'children','charges'],
    outputCol='numeric')
dx=vc.transform(df).select('numeric')

In [22]:
dx.show()

+--------------------+
|             numeric|
+--------------------+
|[19.0,27.9,0.0,16...|
|[18.0,33.77,1.0,1...|
|[28.0,33.0,3.0,44...|
|[33.0,22.705,0.0,...|
|[32.0,28.88,0.0,3...|
|[31.0,25.74,0.0,3...|
|[46.0,33.44,1.0,8...|
|[37.0,27.74,3.0,7...|
|[37.0,29.83,2.0,6...|
|[60.0,25.84,0.0,2...|
|[25.0,26.22,0.0,2...|
|[62.0,26.29,0.0,2...|
|[23.0,34.4,0.0,18...|
|[56.0,39.82,0.0,1...|
|[27.0,42.13,0.0,3...|
|[19.0,24.6,1.0,18...|
|[52.0,30.78,1.0,1...|
|[23.0,23.845,0.0,...|
|[56.0,40.3,0.0,10...|
|[30.0,35.3,0.0,36...|
+--------------------+
only showing top 20 rows



In [23]:
abc=df.toPandas()

In [25]:
abc.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [27]:
df.dtypes

[('age', 'int'),
 ('sex', 'string'),
 ('bmi', 'double'),
 ('children', 'int'),
 ('smoker', 'string'),
 ('region', 'string'),
 ('charges', 'double')]

In [28]:
 cat_cols=[c[0] for c in df.dtypes if c[1]=="string"]

In [29]:
cat_cols

['sex', 'smoker', 'region']

In [30]:
num_cols=[c[0] for c in df.dtypes if c[1]!="string"]
num_cols

['age', 'bmi', 'children', 'charges']

In [31]:
ohe_cols=[c+"_ohe" for c in cat_cols]
ohe_cols

['sex_ohe', 'smoker_ohe', 'region_ohe']

In [32]:
len(cat_cols)

3

In [33]:
cat_cols_si= [c+"_index" for c in cat_cols]
print (cat_cols_si)

['sex_index', 'smoker_index', 'region_index']


In [34]:
cat_cols_ohe= [c+"_ohe" for c in cat_cols]
print (cat_cols_ohe)

['sex_ohe', 'smoker_ohe', 'region_ohe']


In [36]:
df.groupby('region').count().show()

+---------+-----+
|   region|count|
+---------+-----+
|northwest|  325|
|southeast|  364|
|northeast|  324|
|southwest|  325|
+---------+-----+



In [40]:
df.groupby('region').count().orderBy("count", ascending=False).first()[0]

'southeast'

In [41]:
[[i, df.groupby(i).count().orderBy("count", ascending=False).first()[0]] for i in df.columns]

[['age', 18],
 ['sex', 'male'],
 ['bmi', 32.3],
 ['children', 0],
 ['smoker', 'no'],
 ['region', 'southeast'],
 ['charges', 1639.5631]]

## String To Label

In [42]:
lable_To_integer=StringIndexer(
                inputCol="charges",
                outputCol="label"
                )

model=lable_To_integer.fit(df.select("charges"))
model.transform(df.select('charges')).take(10)

[Row(charges=16884.924, label=340.0),
 Row(charges=1725.5523, label=358.0),
 Row(charges=4449.462, label=891.0),
 Row(charges=21984.47061, label=500.0),
 Row(charges=3866.8552, label=783.0),
 Row(charges=3756.6216, label=766.0),
 Row(charges=8240.5896, label=1212.0),
 Row(charges=7281.5056, label=1152.0),
 Row(charges=6406.4107, label=1085.0),
 Row(charges=28923.13692, label=641.0)]

In [43]:
## Pipeline
pipe = Pipeline(
                stages=[
                    StringIndexer( #target column from double to index
                        inputCol="charges",
                        outputCol="label"
                    ),
                    StringIndexer(
                        inputCols=cat_cols,
                        outputCols=cat_cols_si
                    ),
                    OneHotEncoder(
                        inputCols=cat_cols_si,
                        outputCols=ohe_cols
                    ),
                    VectorAssembler(
                        inputCols=cat_cols_si+num_cols,
                        outputCol="ass_features"
                    ),
                    StandardScaler(
                        inputCol="ass_features",
                        outputCol="features"
                    )
                ]
    )

In [44]:
cat_cols

['sex', 'smoker', 'region']

In [45]:
model=pipe.fit(df)

df_trans=model.transform(df)

In [46]:
print (df_trans.columns)

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges', 'label', 'sex_index', 'smoker_index', 'region_index', 'sex_ohe', 'smoker_ohe', 'region_ohe', 'ass_features', 'features']


In [47]:
len(df.columns)

7

In [48]:
len(df_trans.columns)

16

In [49]:
pipe = Pipeline(
                stages=[
                    StringIndexer(
                        inputCol="charges",
                        handleInvalid='keep',#for nan values
                        outputCol="label"
                    ),
                    StringIndexer(
                        inputCols=cat_cols,
                        handleInvalid='keep',
                        outputCols=cat_cols_si
                    ),
                    OneHotEncoder(
                        inputCols=cat_cols_si,
                        outputCols=ohe_cols
                    ),
                    VectorAssembler(
                        inputCols=cat_cols_si+num_cols,
                        outputCol="ass_features"
                    ),
                    StandardScaler(
                        inputCol="ass_features",
                        outputCol="features"
                    ),
                    LogisticRegression(
                        featuresCol='features',
                        labelCol='label',
                        maxIter=30
                    )
                ]
    )

In [50]:
train, test=df.randomSplit([0.8,0.2], seed=12345)

In [51]:
train.count()

1094

In [52]:
test.count()

244

In [53]:
# Training of model
start=time.time()
lrModel=pipe.fit(train)
end=time.time()
(end-start)/60

0.10954660177230835

## Predictions

In [54]:
predictions=lrModel.transform(test)

In [55]:
predictions.columns

['age',
 'sex',
 'bmi',
 'children',
 'smoker',
 'region',
 'charges',
 'label',
 'sex_index',
 'smoker_index',
 'region_index',
 'sex_ohe',
 'smoker_ohe',
 'region_ohe',
 'ass_features',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [56]:
predictions.select("label","prediction","rawprediction","probability").show(5)

+------+----------+--------------------+--------------------+
| label|prediction|       rawprediction|         probability|
+------+----------+--------------------+--------------------+
|1094.0|     600.0|[-96.815634426645...|[7.30152320121927...|
|1094.0|     259.0|[-117.20048655001...|[8.94981321957775...|
|1094.0|      85.0|[-120.48309546859...|[8.62402554252699...|
|1094.0|     415.0|[-141.64279595378...|[1.45504626194398...|
|1094.0|     238.0|[-10.634732577268...|[1.87236863862052...|
+------+----------+--------------------+--------------------+
only showing top 5 rows



# Evaluation Of Model

In [57]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

evaluator.evaluate(predictions)



1.0

In [58]:
evaluator.getMetricName()

'areaUnderROC'

In [59]:
pipe.getStages()

[StringIndexer_ac91b6bd7dc7,
 StringIndexer_ef07c46014ac,
 OneHotEncoder_64977658e4db,
 VectorAssembler_bcfcad9af2ec,
 StandardScaler_cd53ec0f10c2,
 LogisticRegression_0319aae247b6]

# Model Tuning

In [60]:
lr=pipe.getStages()[5]

In [61]:
grid=(ParamGridBuilder().addGrid(lr.regParam, [0.01, 1, 2.0, 5.0]) \
      .addGrid(lr.elasticNetParam, [0.0,0.5,1.0]) \
      .addGrid(lr.maxIter,[1,20]).build()
     
     )

In [62]:
evaluator=BinaryClassificationEvaluator()

cv=CrossValidator(estimator =pipe, estimatorParamMaps=grid, evaluator=evaluator, numFolds=3
              )

In [63]:
cvModel=cv.fit(train)

In [64]:
predictions=cvModel.transform(test)

In [65]:
predictions.select("label","prediction","rawprediction","probability").show(5)

+------+----------+--------------------+--------------------+
| label|prediction|       rawprediction|         probability|
+------+----------+--------------------+--------------------+
|1094.0|     731.0|[-0.0834183480355...|[8.37447846866093...|
|1094.0|     731.0|[-0.0938599059609...|[8.29051039340664...|
|1094.0|     731.0|[-0.0969050319409...|[8.26267023385284...|
|1094.0|     731.0|[-0.1075069237463...|[8.17732442054844...|
|1094.0|     748.0|[-0.0491814989198...|[8.67420648454506...|
+------+----------+--------------------+--------------------+
only showing top 5 rows



In [66]:

evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

evaluator.evaluate(predictions)



1.0